In [3]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.6 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3145188 sha256=066b8caae4557d212e16719d266cf1c4fa93f2733e1014081a3b5fa3a61cabf3
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [4]:
import pandas as pd
import numpy as np
import os
import unicodedata
import torch
import json
import random
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
import unicodedata

In [5]:
YelpZip_file_path = '/content/drive/MyDrive/CSE 573: Fake Reviews Project/YelpZip'

In [6]:
from google.colab import drive
drive.mount('/content/drive')
metadata = pd.read_csv(os.path.join(YelpZip_file_path, 'metadata'), sep= '\t',
                      names=['user_id', 'prod_id','rating','label', 'date'], header=None)

with open(os.path.join(YelpZip_file_path, 'reviewContent')) as r:
  content = r.readlines()

Mounted at /content/drive


In [7]:

number_of_samples = 30000

self_trained_model = fasttext.load_model('/content/drive/MyDrive/CSE 573: Fake Reviews Project/trained_YelpZip_model.bin')

def fasttext_emb(content, metadata,ft_model):
  dataset = []
  label = []
  for text, lab in zip(content[:number_of_samples], metadata[:number_of_samples].itertuples(index=True, name='Pandas')):
    sentence = text.strip('\n').split('\t')[-1]

    sentence = unicodedata.normalize("NFKD", sentence)
    
    sentence_vector = ft_model.get_sentence_vector(sentence)
    dataset.append(sentence_vector)

    if lab.label == -1:
      l = 0
    else: 
      l = 1
    label.append(l)
  
  return dataset, label

In [8]:
dataset,label = fasttext_emb(content, metadata,self_trained_model)
x_train, x_test, y_train, y_test = train_test_split(dataset, label, random_state=42,train_size=0.8) #train 80% test 20%


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [20]:
print(y_train.count(1))
print(y_train.count(0))
print(y_test.count(1))
print(y_test.count(0))

20731
3269
5185
815


In [10]:
rdf = GridSearchCV(
    RandomForestClassifier(),
    param_grid={"n_estimators": [100],"n_jobs":[8]},
    n_jobs= 12
)
rdf.fit(x_train, y_train)
y_pred = rdf.predict(x_test)

In [11]:
print("best param:", rdf.best_estimator_)

best param: RandomForestClassifier(n_jobs=8)


In [12]:
print("accuracy_score: ", accuracy_score(y_test,y_pred))
print("precision_score: ", precision_score(y_test,y_pred, pos_label=1))
print("recall_score: ", recall_score(y_test,y_pred, pos_label=1))
print("f1_score: ", f1_score(y_test,y_pred, pos_label=1))

accuracy_score:  0.8635
precision_score:  0.8686254643701452
recall_score:  0.9920925747348119
f1_score:  0.9262627172053659


In [13]:
print("accuracy_score: ", accuracy_score(y_test,y_pred))
print("precision_score: ", precision_score(y_test,y_pred, pos_label=0))
print("recall_score: ", recall_score(y_test,y_pred, pos_label=0))
print("f1_score: ", f1_score(y_test,y_pred, pos_label=0))

accuracy_score:  0.8635
precision_score:  0.47435897435897434
recall_score:  0.04539877300613497
f1_score:  0.08286674132138859
